In [27]:
#!/usr/bin/env python3

import psycopg2
import pandas as pd
import numpy as np
import time
from preprocesamiento import preprocesamiento


# Put your credentials
conn = psycopg2.connect(
    database="bd2_proyecto",
    host="localhost",
    user="postgres",
    password="postgres",
    port="5432",
)


# Create table
def create_table():
    try:
        cursor = conn.cursor()
        cursor.execute(
            """
            CREATE TABLE IF NOT EXITS songs (
                track_id varchar(22),
                lyrics varchar(27698),
                track_name varchar(123),
                track_artist varchar(51),
                track_album_name varchar(151),
                playlist_name varchar(120),
                playlist_genre varchar(5),
                playlist_subgenre varchar(25),
                language varchar(2),
                info_vector tsvector
        );
        """
        )

        cursor.close()
        conn.commit()
        print("Table created")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


def insert_all(csv_path, columns):
    try:
        cursor = conn.cursor()

        columns_parsed = ", ".join(columns)

        cursor.execute("SELECT COUNT(*) FROM songs")
        count = cursor.fetchone()[0]
        if count == 0:
            df = pd.read_csv(csv_path)
            df["language"] = df["language"].replace({np.nan: None})
            command = f"""
            INSERT INTO songs({columns_parsed})
            VALUES(%s{", %s" * (len(columns) - 1)});
            """
            for _, row in df.iterrows():
                values = tuple(row[col] for col in columns)
                cursor.execute(command, values)

            # Aquí medimos el tiempo con EXPLAIN ANALYZE
            explain_command = f"EXPLAIN ANALYZE {command}"
            cursor.execute(explain_command, values)
            results = cursor.fetchall()
            insertion_time = extract_time(results)

            conn.commit()
            print(f"Tiempo de inserción de datos: {insertion_time} ms")
            return insertion_time
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: ", error)
        conn.rollback()
    finally:
        cursor.close()

def set_index():
    try:
        cursor = conn.cursor()

        # Medir tiempo con clock_timestamp
        command = """
        DO $$ 
        DECLARE
            start_time TIMESTAMP;
            end_time TIMESTAMP;
            duration DOUBLE PRECISION;
        BEGIN
            start_time := clock_timestamp();
            CREATE INDEX IF NOT EXISTS idx_info_vector ON songs USING GIN(info_vector);
            end_time := clock_timestamp();
            duration := EXTRACT(EPOCH FROM (end_time - start_time)) * 1000;  -- Tiempo en ms
            INSERT INTO temp_timing(duration) VALUES (duration);  -- Guardar duración temporalmente
        END $$;
        """
        
        # Crear una tabla temporal para almacenar el tiempo
        cursor.execute("""
        CREATE TEMP TABLE IF NOT EXISTS temp_timing (
            duration DOUBLE PRECISION
        );
        """)
        
        # Ejecutar el bloque DO
        cursor.execute(command)

        # Recuperar el tiempo almacenado
        cursor.execute("SELECT duration FROM temp_timing;")
        index_creation_time = cursor.fetchone()[0]

        print(f"Tiempo de creación del índice: {index_creation_time:.2f} ms")
        return index_creation_time  
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()
    finally:
        cursor.close()




def update_index(language):
    try:
        cursor = conn.cursor()

        command = f"""
        EXPLAIN ANALYZE
        UPDATE songs SET info_vector =
            setweight(to_tsvector('{language}', COALESCE(track_name, '')), 'A') ||
            setweight(to_tsvector('{language}', COALESCE(track_album_name, '')), 'B') ||
            setweight(to_tsvector('{language}', COALESCE(track_artist, '')), 'C') ||
            setweight(to_tsvector('{language}', COALESCE(lyrics, '')), 'D');
        """
        cursor.execute(command)
        results = cursor.fetchall()
        update_time = extract_time(results)
        conn.commit()
        print(f"Info vector updated in {update_time} ms")
        return update_time
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()
        conn.rollback()
    finally:
        cursor.close()


def clean():
    try:
        cursor = conn.cursor()

        drop_index_command = """
        DROP INDEX IF EXISTS idx_info_vector;
        """
        cursor.execute(drop_index_command)
        print("Index dropped (if existed)")

        delete_table_command = """
        DELETE FROM songs;
        """
        cursor.execute(delete_table_command)
        print("Table emptied")

        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()
    finally:
        cursor.close()



def extract_time(time):
    """
    Given the fechall of a explain analyze, it returns the time
    """
    return float(time[-1][0].split(":")[1][1:].split(" ")[0])


def search(query, columns, k=10, time=False):
    """
    if time is true, the executed time of the command is returned
    """
    try:
        # The query is procesed to put the or operators
        query_procesed = preprocesamiento(query)

        # Parse columns list so that they are a string of the selected
        columns_parsed = ", ".join(columns) + ", "

        # Parse columns list so that they are a string of the selected

        cursor = conn.cursor()

        command = f"""
        SELECT {columns_parsed}
               ctid::text as row_position,
               ts_rank_cd(info_vector, {query_procesed}) as score
        FROM songs, to_tsquery({query_procesed}) query
        WHERE info_vector @@ query
        ORDER BY score DESC
        LIMIT {k};
        """
        if time is False:
            cursor.execute(command)
            results = cursor.fetchall()
        else:
            command = "EXPLAIN ANALYZE " + command
            cursor.execute(command)
            results = cursor.fetchall()
            results = extract_time(results)

        # returns a list with tuples, where every tuple is a row
        return results
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()
        conn.rollback()
    finally:
        cursor.close()


# Use if first time creating
# create_table()
# set_index()

# The below code is for inserting the data
clean()
columns = [
    "track_id",
    "lyrics",
    "track_name",
    "track_artist",
    "track_album_name",
    "playlist_name",
    "playlist_genre",
    "playlist_subgenre",
    "language",
]





def obtain_times(queries, columns, k):
    times = []
    for i in queries:
        current_time = search(i, columns, k, True)
        print(current_time)
        times.append(current_time)
    return times




Index dropped (if existed)
Table emptied


In [ ]:

# csv_path = "datasets/spotify_1000.csv"
# insert_all(csv_path, columns)
# update_index("english")

# columns = ["track_id", "track_name", "track_artist", "track_album_name"]
# # Las queries deben estar asi porque con """ hay más espacio, arruinando el
# # preprocesamiento
# queries = [
#     "Don't sweat all the little things Just keep your eye on the bigger things Cause if you look a little closer You're gonna get a bigger picture",
#     "I'mma make your CXRPSE dance Ugh, hop in that Jag, do the dash I shoot a nigga then laugh Bitch, don't talk to me if you ain't on that",
# ]


# k = 20
# times = obtain_times(queries, columns, k)
# for i in range(len(times)):
#     print("Querry i: ", queries[i])
#     print("Elapsed time: ", times[i], "ms")

In [ ]:
# csv_path = "datasets/spotify_1000.csv"

# datasets = ["spotify_1000.csv", "spotify_2000.csv", "spotify_4000.csv"]

# clean()
# #Tiempo de creacion
# set_index_time = set_index()

# insert_time = insert_all(csv_path, columns)

# update_time = update_index("english")

# #Tiempo de querys: 
# columns = ["track_id", "track_name", "track_artist", "track_album_name"]
# queries = [
#     "Don't sweat all the little things Just keep your eye on the bigger things Cause if you look a little closer You're gonna get a bigger picture",
#     "I'mma make your CXRPSE dance Ugh, hop in that Jag, do the dash I shoot a nigga then laugh Bitch, don't talk to me if you ain't on that",
# ]

# k = 20
# times = obtain_times(queries, columns, k)
# for i in range(len(times)):
#     print("Querry i: ", queries[i])
#     print("Elapsed time: ", times[i], "ms")
# clean()



Index dropped (if existed)
Table emptied
Tiempo de creación del índice: 7.03 ms
Tiempo de inserción de datos: 0.045 ms
Info vector updated in 1164.491 ms
548.222
266.311
Querry i:  Don't sweat all the little things Just keep your eye on the bigger things Cause if you look a little closer You're gonna get a bigger picture
Elapsed time:  548.222 ms
Querry i:  I'mma make your CXRPSE dance Ugh, hop in that Jag, do the dash I shoot a nigga then laugh Bitch, don't talk to me if you ain't on that
Elapsed time:  266.311 ms
Index dropped (if existed)
Table emptied


In [37]:
def measure_times_for_datasets(datasets, columns, queries, k, language="english"):
    results = []
    for dataset in datasets:
        csv_path = f"datasets/{dataset}"
        
        clean()
        
        print("Creacion time")

        set_index_time = set_index()  
        insert_time = insert_all(csv_path, columns)
        
        update_time = update_index(language)
        
        print("Queris timecito")
        query_times = obtain_times(queries, columns, k)
        average_query_time = sum(query_times) / len(query_times)
        
        results.append({
            "dataset": dataset,
            "creation_time_s": (set_index_time+insert_time+update_time)/ 1000,
            "average_query_time_s": average_query_time/ 1000,
        })

        df = pd.DataFrame(results)
    
        df.to_csv("tiempos_Postgres_256.csv", index=False)



    return results


# datasets = ["spotify_1000.csv", "spotify_2000.csv", "spotify_4000.csv", "spotify_8000.csv"]
# datasets = ["spotify_16000.csv", "spotify_32000.csv", "spotify_64000.csv"]
datasets = ["spotify_df_256k.csv"]
columns = ["track_id", "track_name", "track_artist", "track_album_name"]
queries = [
    "Don't sweat all the little things Just keep your eye on the bigger things Cause if you look a little closer You're gonna get a bigger picture",
    "I'mma make your CXRPSE dance Ugh, hop in that Jag, do the dash I shoot a nigga then laugh Bitch, don't talk to me if you ain't on that",
]
k = 20

results = measure_times_for_datasets(datasets, columns, queries, k)

for result in results:
    print(result)


Index dropped (if existed)
Table emptied
Creacion time
Tiempo de creación del índice: 4.63 ms
Tiempo de inserción de datos: 0.208 ms
Info vector updated in 13029.339 ms
Queris timecito
9239.846
3718.989
{'dataset': 'spotify_df_256k.csv', 'creation_time_s': 13.034175999999999, 'average_query_time_s': 6.479417499999999}
